# Computation and Simulation

Computation and simulation of robotic systems

### Simulation

-   What are we doing?

-   Why are we simulating?

-   How are we doing this?

### Simulation

Russel Smith’s take:

-   Simulations are quicker to build than robots.  
    But realistic simulations are hard to build: need to model actuators
    (electrical motor dynamics, hydraulics / pneumatics, gear boxes,
    friction, stiction, flexion and slip), sensors, robot geometry and
    mass distribution, joint geometries, flexible bodies (vibration
    effects).

-   Simulations let you cheat.  
    Easy to make controllers that exploit quirks of the simulated world,
    that don’t work so well in real life.

-   Simulated robots do not break.  
    The cost of experimentation may be lower.

-   The best tradeoff:  
    Prototype robot control algorithms on a good-enough simulation, then
    move to the real hardware.

### Simulation

-   API primitives: rigid or flexible bodies, joints, contact with
    friction, collision detection, etc...

-   Many techniques, many libraries, lots of research.

-   Applications:

    -   Game engines

    -   Animation software

    -   Industrial Design

    -   Robotics

-   Research tool.

### Essential Components

1.  Equations of Motion  
    Lagrangian formulations

2.  Integration  
    Numerical quadrature

3.  Contact and Friction  
    Constraints

4.  Optimization  
    To address constrained optimization problem

5.  Collision  
    Collision detection required

In [ ]:
using Plots
using LinearAlgebra
using NLsolve
using Interact
#using Pkg
#Pkg.add("Interact")

### Two link manipulator - Forward

In [ ]:
a1, a2 = 15.0, 15.1
θ1, θ2 = 0.1, 0.2
x = a2*cos(θ1 + θ2) + a1*cos(θ1)
y = a2*sin(θ1 + θ2) + a1*sin(θ1)
println("x = ", x, ", y = ", y)

### Two link manipulator - Inverse

In [ ]:
a1,a2 = 15.0,15.0
x,y = 10.0,8.0
d =  (x*x+y*y-a1*a1-a2*a2)/(2*a1*a2)
println("x = ", x, " y = ", y)
println("d = ", d)

θ2 = atan(-sqrt(1.0-d*d),d)
θ1 = atan(y,x) - atan(a2*sin(θ2),a1+a2*cos(θ2))
println("Angles:  θ1 =", θ1,",  θ2 = ", θ2)

x1 = a2*cos(θ1+θ2) + a1*cos(θ1)
y1 = a2*sin(θ1+θ2) + a1*sin(θ1)
println("Check: x = ", x1, ", y = ", y1)

### Two link manipulator   - Functions

In [ ]:
function twolinkik(x,y)
    d =  (x*x+y*y-a1*a1-a2*a2)/(2*a1*a2)
    θ2 = atan(-sqrt(1.0-d*d),d)
    θ1 = atan(y,x) - atan(a2*sin(θ2),a1+a2*cos(θ2))
    return θ1, θ2
end

function twolinkfk(θ1, θ2)
    x = a2*cos(θ1 + θ2) + a1*cos(θ1)
    y = a2*sin(θ1 + θ2) + a1*sin(θ1)
    return x, y
end

x, y = 10, 8
θ1, θ2 = twolinkik(x, y)
x1, y1 = twolinkfk(θ1, θ2)
println("x = ", x, " y = ", y)
println("Check: x = ", x1, ", y = ", y1)

### Control

There are several aspects of a manipulator we might control:

-   Position Control

-   Velocity Control

-   Force Control

Normally you will have a discrete data to set the control points. For
example: with position control, changes in position are done by sending
new coordinates to the controller. Movement between control points is
done by default system dynamics.

### Equations of motion for manipulators

Assume we have the forward kinematics map: $\xi = \phi(q)$. Motion is
found via the time derivative.
$$\dot{\xi} = D_t\phi(q) = J_\phi(q)\dot{q}$$ where $q$ are the joint
angles and $J$ is the Jacobian.

Note that the Jacobian need not be square or of full rank. Thus an
inverse need not exist. Given $\phi^{-1}=\psi$, $q = \psi(\xi)$, one can
in principle do the same thing
$$\dot{q} = D_t\psi(\xi) = J_\psi(\xi)\dot{\xi}$$

### Chain Rule

Recall that if $w_k = f_k(x,y,z)$ and $x,y,z$ are functions of $t$, then
the chain rule states
$$\dot{w_k} = \frac{\partial f_k}{\partial x}\frac{dx}{dt} +
\frac{\partial f_k}{\partial y}\frac{dy}{dt} +
\frac{\partial f_k}{\partial z}\frac{dz}{dt}$$ where
$k = 1, 2, ... n$.  
For $n=3$
$$\begin{bmatrix} \dot{w}_1\\[8pt]\dot{w}_2\\[8pt]\dot{w}_3\end{bmatrix}
=
\begin{bmatrix}
 \frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} & \frac{\partial f_1}{\partial z}\\[8pt]
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y} & \frac{\partial f_2}{\partial z}\\[8pt]
\frac{\partial f_3}{\partial x} & \frac{\partial f_3}{\partial y} & \frac{\partial f_3}{\partial z}
\end{bmatrix}
\begin{bmatrix} \frac{dx}{dt}\\[8pt]\frac{dy}{dt}\\[8pt]\frac{dz}{dt}\end{bmatrix}$$

### Equations of motion revisited

Assume we have the forward velocity model:
$$\dot{\xi} =  J_\phi(q)\dot{q}$$ where $q$ are state variables and $J$
is the Jacobian of the forward map.

Determine a velocity (speed and direction) $\dot{\xi}$, and then solve
$\dot{\xi} = J_\phi(q)\dot{q}$ for $\dot{q}$.

Thus we have the iterative process

* Define $$\Delta \xi_k = \xi_k - \xi_{k-1}$$

* Solve $$\Delta \xi_k = J_\phi(q_{k-1})h$$

* Set $$q_k = q_{k-1} + h$$

### Least Squares

Let $A = J_\phi(q)$ and $x=h$ and $b = \Delta \xi$ then
$$J_\phi(q)h = \Delta \xi \quad \Rightarrow \quad Ax=b$$ What can be
said when $A$ is rank deficient?

### Forward and Inverse Kinematics

Given joint angles and actuator lengths it is straightforward to compute
end effector position. Thus it is easy to find effector path as a
function of rotations.$$\begin{pmatrix} \theta_1(t), ... , \theta_n(t)
           \end{pmatrix}\to p(t)$$

It is MUCH harder to find the angle functions if you are given the end
effector path: $$p(t) \to \begin{pmatrix} \theta_1(t), ... , \theta_n(t)
           \end{pmatrix}$$

### Manipulator Jacobians

This Notebook gives examples of computations with the Jacobian.

This Notebook uses a couple of packages...

import Pkg ; Pkg.add("NLsolve")

We begin with the Two Link manipulator.  The forward kinematics is 

In [ ]:
function FK(θ1, θ2, a1, a2)
    x = a2*cos(θ1+θ2) + a1*cos(θ1)
    y = a2*sin(θ1+θ2) + a1*sin(θ1)
    return x,y
end

And the Jacobian is 

In [ ]:
function FKJ(θ1, θ2, a1, a2)
    j11 = -a2*sin(θ1+θ2) - a1*sin(θ1)
    j12 = -a2*sin(θ1+θ2)
    j21 = a2*cos(θ1+θ2) + a1*cos(θ1)
    j22 = a2*cos(θ1+θ2)
    J = [ j11 j12 ; j21 j22]
    return J
end

We can see how this works by a simple command:

In [ ]:
J = FKJ(0.1, 0.2, 10, 10)

If joint velocities are [5,10], then the manipulator velocity is

In [ ]:
qdot = [5,10]
v = J*qdot

We can extract the joint velocity from the linear velocity

In [ ]:
inv(J)*v

We can numerically extract the inverse kinematics.  First a couple of measurement functions

In [ ]:
function distance(x1, y1, x2, y2)
    d = sqrt((x1-x2)^2 + (y1-y2)^2)
    return d
end

function size(v1,v2)
    size = sqrt(v1*v1+v2*v2)
    return size
end

Then we can implement the algorithm in the text:

In [ ]:
x = 10
y = 12
θ1 = 0.1
θ2 = 0.2
a1 = a2 = 10
δ,k = 0.1, 0
xc, yc = FK(θ1, θ2, a1, a2)
d = distance(x,y,xc,yc)

while d > .001
    vx = x - xc
    vy = y - yc
    s = δ*size(vx,vy)
    ux = s*vx
    uy = s*vy
    J = FKJ(θ1, θ2, a1, a2)
    u = [ux, uy]
    w = J\u
    θ1 = θ1 + w[1]
    θ2 = θ2 + w[2]
    xc, yc = FK(θ1, θ2, a1, a2)
    d = distance(x,y,xc,yc)
    k +=1
end
θ1 = θ1 - 2*π*trunc(θ1/(2*π))
θ2 = θ2 - 2*π*trunc(θ2/(2*π))
println("k = ", k, " :  θ1 = ",θ1, ",   θ2 = ", θ2, ",  x = ", xc, ",  y = ", yc)

This works, but is really slow (convergence) so we try a Newton based algorithm.  It also needs the Jacobian.  The NLsolve package solves systems of nonlinear equations. Formally, if F is a multivalued function, then this package looks for some vector x that satisfies F(x)=0 to some accuracy.   Using the NLsolve package it is a simple rewrite:

In [ ]:
function f!(F, x)
    F[1] = 10*cos(x[1]+x[2]) + 10*cos(x[1]) - 10
    F[2] = 10*sin(x[1]+x[2]) + 10*sin(x[1]) - 12
end


function j!(J, x)
    J[1, 1] = -10*sin(x[1]+x[2]) - 10*sin(x[1])
    J[1, 2] = -10*sin(x[1]+x[2])
    J[2, 1] = 10*cos(x[1]+x[2]) + 10*cos(x[1])
    J[2, 2] = 10*cos(x[1]+x[2])
end

res = nlsolve(f!, j!, [ 0.5; 0.2])

What is up with the "!".  This is a Julia convention to indicate that the function modifies its arguments but does not affect the function semantics otherwise.   Numerical solutions to equations is a whole subject and we will move on for now.

How about a simple extension to the three link manipulator?  In the following cells we try the same approach.

In [ ]:
function threelink!(F, x)
    a1 = 10
    a2 = 10
    a3 = 10
    F[1] = a3*cos(x[1] + x[2] + x[3]) + a2*cos(x[1] + x[2]) + a1*cos(x[1])
    F[2] = a3*sin(x[1] + x[2] + x[3]) + a2*sin(x[1] + x[2]) + a1*sin(x[1])
    return F
end

function f!(F, x)
    G = threelink!(F, x)
    F[1] = G[1]-20
    F[2] = G[2]-25
    return F
end

In [ ]:
function j!(J,x)
  a1 = 10
  a2 = 10
  a3 = 10
  J[1,1] = -a3*sin(x[1] + x[2] + x[3])- a2*sin(x[1] + x[2]) - a1*sin(x[1]) 
  J[1,2] = -a3*sin(x[1] + x[2] + x[3])- a2*sin(x[1] + x[2])  
  J[1,3] = -a3*sin(x[1] + x[2] + x[3])
  J[2,1] =  a3*cos(x[1] + x[2] + x[3])+ a2*cos(x[1] + x[2]) + a1*cos(x[1]) 
  J[2,2] =  a3*cos(x[1] + x[2] + x[3])+ a2*cos(x[1] + x[2])  
  J[2,3] =  a3*cos(x[1] + x[2] + x[3]) 
  return J
end

In [ ]:
res = nlsolve(f!, j!, [ 0.1; 0.2; 0.3])

What went wrong?  (The other algorithm fails also.)

The problem is that the Jacobian is not square and so not invertable.  

In [ ]:
J = zeros(2,3)
x = zeros(3)
x[1] = 0.1
x[2] = 0.2
x[3] = 0.3
J = j!(J,x)

We need to apply the Pseudoinverse ...  which is done via the SVD:

In [ ]:
R = svd(J, full = true)

You can access each via:

In [ ]:
R.U

In [ ]:
R.S

In [ ]:
R.Vt

We should check if this is correct...

In [ ]:
R.U*Diagonal(R.S)*R.Vt[1:2,:]

In [ ]:
J

The transpose has shorthand here:

In [ ]:
R.U'

The pseudoinverse is done via the SVD ...  $(R.U * R.S * R.Vt)^+ = R.Vt' * R.S^+ * R.U'$ :

In [ ]:
JI = R.Vt'[:,1:2] * inv(Diagonal(R.S)) * (R.U')

There is shorthand for this

In [ ]:
pinv(J)

We can solve $v = Jw$ for w without explicitly constructing the SVD via

In [ ]:
v = [1,1]
w = pinv(J)*v

The Nullspace is given by the rows corresponding to zero singular values (missing from S):

In [ ]:
N = R.Vt[3,:]

This can be done without explicit construction of the SVD and there is shorthand ...

In [ ]:
nullspace(J)

In [ ]:
F = zeros(2)
x = zeros(3)

x=[0.2, 0.3, 0.4]
δ = 0.01
F = threelink!(F,x)
print("q = ")
println(x)
print("p = ")
println(F, "\n")

x1 = x
for i in 1:10
    J = j!(J,x1)
    v = nullspace(J)
    x1 = x1 + δ*v
end
F = threelink!(F,x1)
print("q = ")
println(x1)
println(norm(x - x1))
print("p = ")
println(F, "\n")

x2 = x + pinv(J)*[-2,3]
F = threelink!(F,x2)
print("q = ")
println(x2)
println(norm(x - x2))
print("p = ")
println(F)

We can put this together...

In [ ]:
function arm(θ1,θ2,θ3)
    x1 = 10*cos(θ1)
    y1 = 10*sin(θ1)
    x2 = x1 + 10*cos(θ1+θ2)
    y2 = y1 + 10*sin(θ1+θ2)
    x3 = x2 + 10*cos(θ1+θ2+θ3)
    y3 = y2 + 10*sin(θ1+θ2+θ3)
    return x1,x2,x3,y1,y2,y3
end

In [ ]:
N = 100
F = zeros(2)
x = zeros(3)
w = zeros(3)
J = zeros(2,3)
q1 = zeros(N)
q2 = zeros(N)
q3 = zeros(N)
p1 = zeros(N)
p2 = zeros(N)
x = [0, 2*π/3, -3π/4]
F = threelink!(F, x)

for i in 1:N
    J = j!(J,x)
    v = 0.1*[-sin(i/N), 2*cos(i/N)]
    w = pinv(J)*v
    x = x + w
    F = threelink!(F, x)
    q1[i] = x[1]
    q2[i] = x[2]
    q3[i] = x[3]
    p1[i] = F[1]
    p2[i] = F[2]
end

In [ ]:
plt = plot(legend=false,xlim=(0,3),ylim=(0,3),aspect_ratio=:equal)
display(plt)
for i = 1:N
    IJulia.clear_output(true)
    x1,x2,x3,y1,y2,y3 = arm(q1[i],q2[i],q3[i])
    l1 = [0,x1,x2,x3]
    l2 = [0,y1,y2,y3]
    plt = plot(l1,l2, legend=false,xlim=(0,30),ylim=(0,30), linewidth=8, aspect_ratio=:equal)
    plot!(p1[1:i],p2[1:i])
    display(plt)
    sleep(.02)
end

### How can we animate?

In [ ]:
N = 4
θ1 = [0.1, .3, .5, 0.8]
θ2 = [0.2, .4, .6, 0.9]
x = a2.*cos.(θ1 .+ θ2) .+ a1.*cos.(θ1)
y = a2.*sin.(θ1 .+ θ2) .+ a1.*sin.(θ1)
a1, a2 = 1, 1
xmid = a1 .* cos.(θ1)
ymid = a1 .* sin.(θ1)
p = scatter(xlim = (-0.1,2.1), ylim = (-0.1,2.1), axis = false, ticks = false, aspect_ratio = :equal, legend = false, color = :blue)
for i = 1:N
    xl = [0, xmid[i],x[i]]
    yl = [0, ymid[i],y[i]]
    plot!(xl,yl, color=:blue,  linewidth=8)
    scatter!(xl, yl, color=:red, markershape=:circle, ms = 6)
end
plot!([1.75,1.35],[1.25,1.65],arrow=true,color=:black,linewidth=2,label="")
display(p)

### Two Link Simulations

For the arm in the two link example, determine the joint angles to trace
out a circle centered at (10,8) of radius 5. The circle can be
parametrized by $x(t) = 5\cos (t) + 10$, $y(t) = 5 \sin(t) + 8$,
$0 \leq t \leq 2\pi$. Generate an array of points on the circle and plug
them into the inverse kinematics.

How can we achieve this ...

Here is an example using the macro @

In [ ]:
t = range(0, 2*π, length = 100)
x = @. 5 * cos(t) + 10.0
y = @. 5 * sin(t) + 8.0
a1,a2 = 15.0,15.0
d =  @. ((x*x) + (y*y) - (a1*a1) - (a2*a2))/(2.0 * (a1*a2))
t2 = @. atan(-sqrt(1.0 - (d*d)),d)
t1 = @. atan(y,x) - atan(a2*sin(t2), a1 + a2*cos(t2))

p = plot(t1,t2,aspect_ratio = :equal,legend=false)
println("Done")

In [ ]:
display(p)

In [ ]:
x1 = @. a2*cos(t1+t2) + a1*cos(t1)
y1 = @. a2*sin(t1+t2) + a1*sin(t1)
p = plot(x1,y1,aspect_ratio = :equal,legend=false)
println("Done")

In [ ]:
display(p)

In [ ]:
function ik(x, y, a1, a2)
   d =  (x*x+y*y-a1*a1-a2*a2)/(2*a1*a2)
   θ2 = atan(-sqrt(1.0-d*d),d)
   θ1 = atan(y,x) - atan(a2*sin(θ2),a1+a2*cos(θ2))
   return (θ1,θ2)
end


function fk(θ1, θ2, a1, a2)
    x = a2*cos(θ1+θ2) + a1*cos(θ1)
    y = a2*sin(θ1+θ2) + a1*sin(θ1)
    return x,y
end

a1, a2 = 15.0, 15.0

t1 = zeros(10)
t2 = zeros(10)

for i = 1:10
    global a1, a2, t1, t2
    x = 9 + 0.1*i
    y = 7 + 0.1*i
   (t1[i], t2[i]) = ik(x, y, a1, a2)
   x1, y1 = fk(t1[i], t2[i], a1, a2)
   println((x-x1)^2 + (y-y1)*2)
end
p=plot(t1,t2)
println("Done")

In [ ]:
display(p)

In [ ]:
# Set the link lengths
L0 = 8
L1 = 5
L2 = 10

# Initialize the arrays
xlist = zeros(0)
ylist = zeros(0)

# Loop over the two angles,
#  stepping about 1.8 degrees each step

for i = 1:100
   for j = 1:100
      th1 = 0 + 1.57*i/100.0
      th2 = 0 + 1.57*j/100.0

      a = -L1*cos(th1) - L0/2.0
      b = L1*sin(th1)
      c = L1*cos(th2) + L0/2.0
      d = L1*sin(th2)

      dx = c-a
      dy = b-d
      u = sqrt(dx*dx+dy*dy)
      v = sqrt(L2*L2 - 0.25*u*u)

      x = (a+c)/2.0 + v*dy/u
      y = (b+d)/2.0 + v*dx/u
      push!(xlist, x)
      push!(ylist, y)
  end
end

p = scatter(xlist,ylist,legend= false,markersize=5)
println("Done")

In [ ]:
display(p)

Using implicit looping without the macro @

In [ ]:
using Plots

# Set the link lengths
L0 = 8
L02 = L0/2.0
L1 = 5
L2 = 10

t = LinRange(0, 1.57, 100)
th1 = t' .* ones(100)
th2 = ones(100)' .* t
a = -L1 .* cos.(th1) .- L02
b = L1 .* sin.(th1)
c = L1 .* cos.(th2) .+ L02
d = L1 .* sin.(th2)
dx = c.-a
dy = b.-d
u = sqrt.((dx .* dx) .+ (dy .* dy))
v = sqrt.((L2 .* L2) .- (0.25.*u.*u))
x = (a.+c)./2.0 .+ (v.*dy./u)
y = (b.+d)./2.0 .+ (v.*dx./u)
p = scatter(x,y,legend= false,markersize=5,markercolor="blue")
println("Done")

In [ ]:
display(p)

###  Interactive Code Examples

This Notebook is here to provide some examples of simulations and interactive code.   

The first cell imports the libraries.  First time use you will need to add the package.  The commented code gives an example for adding the Interact package.

If you have a series of joint values, here is how you might compute the end-effector location and list the values.

In [ ]:
a1, a2 = 10, 10
θ1 = range(π/4, π/3, length = 10)
θ2 = range(π/6, π/4, length = 10)
for i in 1:10
    x = a2*cos(θ1[i]+θ2[i]) + a1*cos(θ1[i])
    y = a2*sin(θ1[i]+θ2[i]) + a1*sin(θ1[i])
    println("x = ", x, ", y = ", y)
end

Normally we want to provide $(x,y)$ location data and the compute the joint values.   So the inverse kinematics is first.  In this next example we will use the implicit loop notation (the dot).  

In [ ]:
N = 50
t = range(1, 5, length = N)
x =  t .+ 1.0
y = 2 .* t .- 1.0
p=plot(x,y, title="Desired Path", aspect_ratio = :equal, legend=false)
println("Done")

In [ ]:
display(p)

Taking these input $(x,y)$ values, we can plug into the IK and obtain the values for $\theta_1$, $\theta_2$.

In [ ]:
a1,a2 = 10.0,10.0
d =  ((x.*x) .+ (y.*y) .- (a1.*a1) .- (a2.*a2))/(2.0 .* (a1.*a2))
θ2 = atan.(-sqrt.(1.0 .- (d.*d)),d)
θ1 = atan.(y,x) - atan.(a2.* sin.(θ2), a1 .+ a2.*cos.(θ2))
p = plot(θ1,θ2, title="Plot of θ1, θ2", aspect_ratio = :equal, legend = false)
println("Done")

In [ ]:
display(p)

The obvious question, is this correct?  An easy way to figure this out is to plug those values into the forward kinematics and plot the results.

In [ ]:
x1 = a2.*cos.(θ1 .+ θ2) .+ a1.*cos.(θ1)
y1 = a2.*sin.(θ1 .+ θ2) .+ a1.*sin.(θ1)
p = plot(x1,y1, title = "Checking Path", aspect_ratio = :equal, legend=false)
# if you want to overlap the plots
# plot!(x,y)
println("Done")

In [ ]:
dislplay(p)

It can be helpful to visualize the dynamics of the manipulator.  The following example is Julia/Plots animation of the two link manipulator endpoints.   An animation needs a delay (the sleep function) and you need this clear output method to replot over the previous plot.

In [ ]:
for i = 1:N
    IJulia.clear_output(true)
    p = scatter([x[i]],[y[i]], xlim = (0,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :green)
    display(p)
    sleep(0.05)
end

If you want to leave the path (the trace), you can try the following variant.

In [ ]:
scatter([x[1]],[y[1]], xlim = (0,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :green)
for i = 2:N
    IJulia.clear_output(true)
    xl = x[1:i]
    yl = y[1:i]
    p = scatter(xl,yl, xlim = (0,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :green)
    display(p)
    sleep(0.05)
end

Just to play with the graphics, we change the trace.  plot! and scatter! are different functions than plot and scatter.  The "!" means this version will add to the previous plot.   Otherwise a new plot is created.

In [ ]:
scatter([x[1]],[y[1]], xlim = (0,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :green)
for i = 2:N
    IJulia.clear_output(true)
    xl = x[1:i]
    yl = y[1:i]
    p = scatter([x[i]],[y[i]], xlim = (0,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :green)
    plot!(xl,yl, color=:red)
    display(p)
    sleep(0.05)
end

An actual animation should in include the link arms.

In [ ]:
xmid = a1 .* cos.(θ1)
ymid = a1 .* sin.(θ1)
scatter([x[1]],[y[1]], xlim = (-10,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :blue)
for i = 2:N
    IJulia.clear_output(true)
    p = scatter([x[i]],[y[i]], xlim = (-10,10), ylim = (0,10), aspect_ratio = :equal, legend = false, color = :blue)
    xl = [0, xmid[i], x[i]]
    yl = [0, ymid[i], y[i]]
    plot!(xl,yl, color=:blue,  linewidth=8)
    scatter!(xl, yl, color=:red, markershape=:circle)
    display(p)
    sleep(0.05)
end

The Interact package connects up some Javascript widgets in the Notebook with Julia.   It supports a variety of widgets and manages the callbacks for you.   This is not a tutorial on the Interact package.  There are some macros available that make the interact package easy to use.  This example sets up two slider bars which are used to set the $\theta_1$, $\theta_2$ values.   

The @manipulate macro sets up the event loop and connects the slider values to values that can be used in the event loop.

In [ ]:
function arm(θ1,θ2)
    x1 = cos(θ1)
    y1 = sin(θ1)
    x2 = x1 + cos(θ1+θ2)
    y2 = y1 + sin(θ1+θ2)
    return x1,x2,y1,y2
end

In [ ]:
s1 = slider(-π:0.05:π ,value = 0.0, label="Theta1")
s2 = slider(-π:0.05:π, value = 0.0, label="Theta2")

mp = @manipulate for θ1 in s1, θ2 in s2
    x1,x2,y1,y2 = arm(θ1,θ2)
    xl = [0,x1,x2]
    yl = [0,y1,y2]
    plot(xl,yl, legend=false,xlim=(-2,2),ylim=(-2,2), aspect_ratio = :equal, linewidth=8)
    scatter!(xl, yl, color=:red, markershape=:circle)
end

To demonstrate how this can be used in 3D, here is the manipulator from the last homework (#23).  

In [ ]:
function arm3(d, a1, a2, θ1)
    x1 = 0
    y1 = 0
    z1 = d
    x2 = a1*cos(θ1)
    y2 = a1*sin(θ1)
    z2 = z1
    x3 = x2
    y3 = y2
    z3 = z1 - a2
    j = [x1,y1,z1,x2,y2,z2,x3,y3,z3]
    return j
end

This gives an example of plots in 3D.

In [ ]:
s1 = slider(0.0:0.01:π/2 ,value = 0.0, label="Theta1")
s2 = slider(1.0:0.01:5, value = 2.0, label="a1")
s3 = slider(1.0:0.01:5, value = 3.0, label="a2")
d = 5

mp = @manipulate for θ1 in s1, a1 in s2, a2 in s3
    j = arm3(d,a1,a2,θ1)
    p1 = [0, j[1], j[4], j[7]]
    p2 = [0,  j[2], j[5], j[8]]
    p3 = [0, j[3], j[6], j[9]]
    plot(p1,p2,p3, xlim=(0,6),ylim=(0,6),zlim=(0,6),linewidth=10,legend=false)
end

A simple "Etch-a-Sketch" type demo:

In [ ]:
s1 = slider(-1:0.1:1, value = 0.0, label="x")
s2 = slider(-1:0.1:1, value = 0.0, label="y")
plot(legend=false,xlim=(-1.5,1.5),ylim=(-1.5,1.5))

mp = @manipulate for x in s1, y in s2
    l1 = [x]
    l2 = [y]
    plot!(l1,l2, markershape=:circle, markercolor=:blue)
end

An interactive plotting tool:

In [ ]:
x = range(0, 10, length=100)
y = sin.(x) .+ 1.5

s1 = slider(1:100, value = 1, label="time")

scatter(legend=false,xlim=(0,10),ylim=(0,3))

mp = @manipulate for t in s1
    i = trunc(Int,t)
    l1 = x[1:i]
    l2 = y[1:i]
    scatter!(l1,l2, markershape=:circle, markercolor=:blue)
end

In [ ]:
x = y = 0:0.1:30

freqs = OrderedDict(zip(["pi/4", "π/2", "3π/4", "π"], [π/4, π/2, 3π/4, π]))

mp = @manipulate for freq1 in freqs, freq2 in slider(0.01:0.1:4π; label="freq2")
    y = @. sin(freq1*x) * sin(freq2*x)
    plot(x, y)
end

An example showing how to clear a plot.

In [ ]:
x = range(0, 10, length=100)
y = sin.(x) .+ 1.5

s1 = slider(1:100, value = 1, label="Time")
s2 = OrderedDict(zip(["Plot", "Clear"], [1, 0]))

scatter(legend=false,xlim=(0,10),ylim=(0,3))

mp = @manipulate for t in s1, Select in s2
    i = trunc(Int,t)
    if Select == 0
        scatter(legend=false,xlim=(0,10),ylim=(0,3))
    else
        l1 = x[1:i]
        l2 = y[1:i]
        scatter!(l1,l2, markershape=:circle, markercolor=:blue)
    end
end

### One can have a few issues using only position control

In [ ]:
using Plots
N = 100
t = range(0, 2*π, length = N)
x = @. 5 * cos(t) + 10.0
y = @. 5 * sin(t) + 8.0
p1 = plot(x,y, aspect_ratio=:equal, title="Requested Path")

a1,a2 = 15.0,15.0
d =  @. ((x*x) + (y*y) - (a1*a1) - (a2*a2))/(2.0 * (a1*a2))
t2 = @. atan(-sqrt(1.0 - (d*d)),d)
t1 = @. atan(y,x) - atan(a2*sin(t2), a1 + a2*cos(t2))
p2 = plot(t1,t2,aspect_ratio=:equal, title="Joint Angles")

x1 = @. a2*cos(t1+t2) + a1*cos(t1)
y1 = @. a2*sin(t1+t2) + a1*sin(t1)
p3 = plot(x1,y1,aspect_ratio=:equal, title="Computed Path")

x2 = @. a2*cos(t1+t2) + a1*cos(t1)
y2 = @. a2*sin(t1+t2) + a1*sin(t1)
p4 = scatter(x2,y2,aspect_ratio=:equal, title="Points")
println("Plots generated")

In [ ]:
plot(p1,p2,p3,p4, layout = (2,2), legend = false)

This is what will actually happen...  

L is the number of total points around the circle.   N is the number of computed postion points.
M is the points that the controller will set (using a simple servo stepping approach).

In [ ]:
using Plots
L = 500
N = 20
M = floor(Int64, L/N)
t = range(0, 2*π, length = N)
x = @. 5 * cos(t) + 10.0
y = @. 5 * sin(t) + 8.0
p1 = scatter(x,y, aspect_ratio=:equal, title="Requested Path", ms = 1)

a1,a2 = 15.0,15.0
d =  @. ((x*x) + (y*y) - (a1*a1) - (a2*a2))/(2.0 * (a1*a2))
t2 = @. atan(-sqrt(1.0 - (d*d)),d)
t1 = @. atan(y,x) - atan(a2*sin(t2), a1 + a2*cos(t2))
p2 = scatter(t1,t2,aspect_ratio=:equal, title="Joint Angles", ms = 1)

x1 = @. a2*cos(t1+t2) + a1*cos(t1)
y1 = @. a2*sin(t1+t2) + a1*sin(t1)
p3 = scatter(x1,y1,aspect_ratio=:equal, title="Computed Path", ms = 1)

x2 = @. a2*cos(t1+t2) + a1*cos(t1)
y2 = @. a2*sin(t1+t2) + a1*sin(t1)
x3 = []
y3 = []
for i in 1:N-1
    θ1 = range(t1[i], t1[i+1],length=M)
    θ2 = range(t2[i], t2[i+1],length=M)
    for j in 1:M
        tx = a2*cos(θ1[j]+t2[i]) + a1*cos(θ1[j])
        ty = a2*sin(θ1[j]+t2[i]) + a1*sin(θ1[j])
        push!(x3,tx)
        push!(y3,ty)
    end
    for j in 1:M
        tx = a2*cos(t1[i+1]+θ2[j]) + a1*cos(t1[i+1])
        ty = a2*sin(t1[i+1]+θ2[j]) + a1*sin(t1[i+1])
        push!(x3,tx)
        push!(y3,ty)        
    end
end      
p4 = plot(x3,y3,aspect_ratio=:equal, title="Traversal", ms = 1)
println("Plots generated")

In [ ]:
plot(p1,p2,p3,p4, layout = (2,2), legend = false)

## Differential Drive

Computing trajectories of vehicles

### Problems with Differential Drive Simulations and computations

Problems:

1.  What if you can’t integrate the function to get $\theta$?

2.  With $\theta$ determined, what if you can’t integrate the $x$ and
    $y$ formulas?

Example: Let $\dot{\phi_1} = e^{-t^2}$ and $\dot{\phi_2} = t$
$$\theta(t) = \theta(0) + \int_0^t \frac{r}{2L} \left(e^{-\tau^2}-\tau\right)d\tau = ???$$
Now what does one do? There is another problem as well.

### Simulation of motion - velocity control - Example

Take an example:
$$\displaystyle \left(\frac{dx}{dt}, \frac{dy}{dt}\right) =
\left\{
\begin{array}{ll}
(0.5t, 0.0),  & 0 \leq t < 2, \\[3mm]
(0.25t, 0.65t),  & 2 \leq t < 5,
\end{array}
\right.$$

How should we do this?

### Simulation of motion - velocity control

The motion equations:
$$\displaystyle \frac{dx}{dt} = f(t), \quad\quad \displaystyle \frac{dy}{dt} = g(t)$$
where $f$ and $g$ are the input/control velocities.

What we do in the simulation is a discrete approximation of the world -
the digital world is a discrete sampled world.

### Simulation of motion - velocity control

Arrays of velocities, $[f_1, f_2, f_3, ...]$ and $[g_1,g_2,g_3, ...]$.

Using the approximation of a derivative:
$$\displaystyle \frac{dx}{dt} \approx \frac{x_{i+1} - x_i}{\Delta t}$$
where $\Delta t$ is the time between samples.

We can express the formula for the change in distance based on this
velocity update:
$$x_{i+1} = x_i + f_i \Delta t  , \quad\quad y_{i+1} = y_i + g_i \Delta t .$$
Now the transition from $x_i$ to $x_{i+1}$ follows the physics more
closely (assuming you have realistic velocities).

### Discrete approximation

What if you don’t have $\phi(t)$ and are measuring the wheel angular
velocity during runtime, AND it is clearly not constant?  
We will use Euler’s method for solving the differential equations. Let
the time between measurements be denoted by $\Delta t$. The discretized
variables are $$t_k \equiv k\Delta t, \quad t_{k+1} = (k+1)\Delta t$$
$$x_k \equiv x(t_k),  y_k \equiv y(t_k)$$
$$\omega_{1, k}\equiv \dot{\phi}_{1}(t_k),
\omega_{2, k}\equiv \dot{\phi}_{2}(t_k)$$

### Discrete approximation

Recall that if $x$ is position then $\dot{x}$ is velocity (and
$\ddot{x}$ is acceleration).  
One may approximate a derivative by
$$\dot{x} \approx \frac{x(t+\Delta t) - x(t)}{\Delta t}$$ Take a time
step of $\Delta t$ (meaning $t_{k+1} = t_k + \Delta t$), Euler’s
(“Oil-ler’s”) method is
$$x(t_{k+1}) = x(t_k) + (\Delta t)x'(t_k) \quad \mbox{and}
\quad y(t_{k+1}) = y(t_k) + (\Delta t)y'(t_k).$$ And so we can write our
differential equations as difference equations...

### Discrete approximation

$$\begin{array}{l}
\displaystyle \frac{x(t+\Delta t) - x(t)}{\Delta t}\approx \dot{x} = \frac{r}{2} (\dot{\phi_1}+\dot{\phi_2})\cos(\theta) \\[5mm]
\displaystyle \frac{y(t+\Delta t) - y(t)}{\Delta t}\approx \dot{y} = \frac{r}{2} (\dot{\phi_1}+\dot{\phi_2})\sin(\theta) \\[5mm]
\displaystyle \frac{\theta (t+\Delta t) - \theta (t)}{\Delta t}\approx \dot{\theta} = \frac{r}{2L} (\dot{\phi_1}-\dot{\phi_2})
\end{array}$$ 

Algebra: $$\begin{array}{l}
 x(t+\Delta t) \approx x(t) +\frac{r\Delta t}{2} (\dot{\phi_1}+\dot{\phi_2})\cos(\theta) \\[5mm]
 y(t+\Delta t) \approx y(t) +\frac{r\Delta t}{2} (\dot{\phi_1}+\dot{\phi_2})\sin(\theta) \\[5mm]
\theta (t+\Delta t) \approx \theta (t) +\frac{r\Delta t}{2L} (\dot{\phi_1}-\dot{\phi_2})
\end{array}$$

### Discrete Differential Drive Model

Using the discrete (sample) variables, $x(t) \to x_k$, etc, we can
rewrite the expression in terms of the discrete variables. Given
starting configuration and wheel velocity measurements, we have:
$$\begin{array}{l}
 x_{k+1} = x_k + \frac{r\Delta t}{2} (\omega_{1, k}+\omega_{2, k})\cos(\theta_k) \\[5mm]
y_{k+1} = y_k + \frac{r\Delta t}{2} (\omega_{1, k}+\omega_{2, k})\sin(\theta_k) \\[5mm]
\theta_{k+1} = \theta_k + \frac{r\Delta t}{2L} (\omega_{1, k}-\omega_{2, k})
\end{array}$$

### How do we do this?

In [ ]:
function DDstep(θ, r, L, ϕ1dot, ϕ2dot, dt)
    δx = (r*dt/2)*(ϕ1dot+ϕ2dot)*cos(θ)
    δy = (r*dt/2)*(ϕ1dot+ϕ2dot)*sin(θ)
    δθ = (r*dt/(2*L))*(ϕ1dot-ϕ2dot)
    return δx, δy, δθ
end

Variable setup for the simulation.

In [ ]:
using Plots
r = 1
L = 2
N = 100
t = range(0, 5, length = N)
ω1 = 1.25 .+ cos.(t)
ω2 = 1.0 .+ sin.(t)
dt = 5/N
x, y = 0, 0
θ = 0

In [ ]:
lx = zeros(N)
ly = zeros(N)
lθ = zeros(N)

for i = 1:(N-1)
    δx, δy, δθ = DDstep(lθ[i], r, L, ω1[i], ω2[i], dt)
    lx[i+1] = lx[i] + δx
    ly[i+1] = ly[i] + δy
    lθ[i+1] = lθ[i] + δθ
end
p = scatter(lx,ly, xlim = (0,12), ylim = (-1,2.5), legend = false, color = :blue)
println("Done")

In [ ]:
display(p)

The animation of the simulation loop

In [ ]:
for i = 1:N
    global x, y, θ
    δx, δy, δθ = DDstep(θ, r, L, ω1[i], ω2[i], dt)
    x = x + δx
    y = y + δy
    θ = θ + δθ
    p = scatter([x],[y], xlim = (0,12), ylim = (-1,3), legend = false, color = :blue)
    display(p)
    sleep(0.2)
    IJulia.clear_output(true)
end

### Guidance

Moving the robot ...

### Differential Drive Model

Recall the differential drive model $$\begin{array}{l}
 \dot{x} = \frac{r}{2} (\dot{\phi_1}+\dot{\phi_2})\cos(\theta) \\[5mm]
\dot{y} = \frac{r}{2} (\dot{\phi_1}+\dot{\phi_2})\sin(\theta) \\[5mm]
\dot{\theta} = \frac{r}{2L} (\dot{\phi_1}-\dot{\phi_2})
\end{array},$$

How does one set the wheel speeds to obtain the desired motion in the
plane?

### Driving the Differential Drive Model

Similarly how does one obtain desired motion with the discrete model:
$$\begin{array}{l}
 x_{k+1} = x_k + \frac{r\Delta t}{2} (\omega_{1, k}+\omega_{2, k})\cos(\theta_k) \\[5mm]
y_{k+1} = y_k + \frac{r\Delta t}{2} (\omega_{1, k}+\omega_{2, k})\sin(\theta_k) \\[5mm]
\theta_{k+1} = \theta_k + \frac{r\Delta t}{2L} (\omega_{1, k}-\omega_{2, k})
\end{array}$$

### Background on Parametric Curves

How does one follow a curve given in function form: $y=f(x)$. First, you
need to write in parametric form: $x(t)$, $y(t)$. This is easily done by
$$\mbox{Let } x = t  \quad \to \quad y = f(x) = f(t)$$ Example: convert
$y=x^2$ to parametric
$$\mbox{Let } x = t  \quad \to \quad y = x^2 = t^2$$ Note that there are
an infinite number of choices:
$$\mbox{Let } x = ct  \quad c\mbox{~nonzero}\quad \to \quad y = x^2 = c^2t^2$$
$$\mbox{Let } x = ce^t  \quad c\mbox{~nonzero}\quad \to \quad y = x^2 = e^{2t}$$
etc ...

### Functional vs Parametric form

All the parametric forms provide the same curve, same shape, same
geometry. They vary in the speed. Think of the function form tells you
the shape - like the shape of a road. The parametric form gives you
shape and velocity - includes how fast you drive the road.

### Speed

$x(t) = \phi(t)$, $y(t) = \psi(t)$ such that
$$v(t) = \sqrt{\dot{\phi}^2 + \dot{\psi}^2}$$

Note the tangent vector
$T = <\dot{x} , \dot{y} > = <\dot{\phi} , \dot{\psi} >$.

The speed is given by
$$\| T \|= \|<\dot{x} , \dot{y} >\|  = \sqrt{\dot{\phi}^2 + \dot{\psi}^2 }$$
where $\| \cdot \|$ is the length or norm of the vector.

Recall that the Inverse Kinematics are
$$
  \dot{\phi_1} =  \frac{L\dot{\theta}}{r} \pm \frac{v}{r}, \quad
   \dot{\phi_2} = -\frac{L\dot{\theta}}{r} \pm \frac{v}{r}
$$
which is
$$
  \dot{\phi_1} = \displaystyle \frac{L}{r}\left( \frac{\dot{x}\ddot{y} - \dot{y}\ddot{x}}{\dot{x}^2 + \dot{y}^2}\right) \pm \frac{\sqrt{\dot{x}^2 + \dot{y}^2}}{r}
  $$
  $$
   \dot{\phi_2} = \displaystyle -\frac{L}{r}\left(\frac{\dot{x}\ddot{y} - \dot{y}\ddot{x}}{\dot{x}^2 + \dot{y}^2}\right) \pm \frac{\sqrt{\dot{x}^2 + \dot{y}^2}}{r}
$$
or written in terms of the curve parameters
$$
 \boxed{
   \begin{array}{l}
   v = \sqrt{\dot{x}^2 + \dot{y}^2}\\[3mm]
   \kappa =   \displaystyle  \frac{\dot{x}\ddot{y} - \dot{y}\ddot{x}}{v^3} = \frac{\dot{\theta}}{v}\\[3mm]
   \dot{\phi_1} = \displaystyle \frac{v}{r}\left(\kappa L + 1\right) \\[3mm]
   \dot{\phi_2} = \displaystyle \frac{v}{r}\left(-\kappa L + 1\right)
   \end{array}}
$$

### Example Code

In [ ]:
N=200
t0 = 0.0
t1 = 2.0
t = range(t0,t1,length =N)
y = zeros(N)
x = t.*t
y = t
p = plot(x,y, legend=false)
println("Done")

In [ ]:
display(p)

### Example Code

In [ ]:
doty=ones(N)
dotx=2.0*t
ddoty=zeros(N)
ddotx=2.0*ones(N)

r = 1.0
L = 4.0
v = @. sqrt(dotx*dotx + doty*doty)
kappa = @. (dotx*ddoty - doty*ddotx)/(v*v*v)
dotphi1 = @. (v/r)*(kappa*L +1)
dotphi2 = @. (v/r)*(-kappa*L+1)
plot(t,dotphi1)
p=plot!(t,dotphi2)
println("Done")

In [ ]:
display(p)

### Example Code

In [ ]:
xp = zeros(N)
yp = zeros(N)
th = zeros(N)
th[1] = π/2
Δt = (t1-t0)/N
δ = r*Δt/2.0
for i in 1:N-1
    xp[i+1] = xp[i] + δ*(dotphi1[i]+dotphi2[i])*cos(th[i])
    yp[i+1] = yp[i] + δ*(dotphi1[i]+dotphi2[i])*sin(th[i])
    th[i+1] = th[i] + δ*(dotphi1[i]-dotphi2[i])/L
end
plot(x,y)
p = scatter!(xp,yp)
println("Done")

In [ ]:
display(p)

### Following a Path

Given a discrete set of points, there are an infinite set of curves that
interpolate the points.

A very simple approach is to connect the points with line segments. This
is known as linear interpolation.

Drive straight, stop, rotate and drive straight again. Repeat.

Problems with this?

### Freeway Lane Change

Although you may think that the drivers in your region are bad …

This is nothing like having each one slam on their brakes on the
freeway, stop, turn and drive to the new lane and stop.

Then they turn and floor it back up to speed. Complete chaos.

The linear interpolant approach does not work for any system that
requires smooth fluid motion.

### Cubic Splines

Assume that you have two points  
$t_0: (x_0,y_0)$ and $t_1: (x_1, y_1)$.

Also assume that you have the tangent (or the derivative) at each
point:  
$t_0: (\dot{x}_0, \dot{y}_0)$ and $t_1: (\dot{x}_1, \dot{y}_1)$.

### Cubic Splines

The cubic spline is
$$x(t) = (1-z)x_0 + z x_1 + z(1-z)\left[ a(1-z) +b z\right]$$
$$y(t) = (1-z)y_0 + z y_1 + z(1-z)\left[ c(1-z) +d z\right]$$ where
$$a = \dot{x}_0(t_1-t_0)-(x_1-x_0), \quad b = -\dot{x}_1(t_1-t_0)+(x_1-x_0)$$
$$c = \dot{y}_0(t_1-t_0)-(y_1-y_0), \quad d = -\dot{y}_1(t_1-t_0)+(y_1-y_0)$$
$$z = \displaystyle \frac{t - t_0}{t_1-t_0}$$

### Cubic Spline Formulas for $t_0 \leq t \leq t_1$:

$$\displaystyle \frac{dx}{dt} =  \frac{dx}{dz}  \frac{dz}{dt} = \left\{x_1 -x_0  + (1 - 2z)\left[ a(1-z) +b z\right] + z(1-z)\left[ b-a\right]\right\}\dot{z}$$
$$\displaystyle \frac{dy}{dt} =  \frac{dy}{dz}  \frac{dz}{dt} = \left\{y_1 -y_0  + (1 - 2z)\left[ c(1-z) +d z\right] + z(1-z)\left[ d-c\right]\right\}\dot{z}$$
$$\displaystyle \frac{d^2x}{dt^2} =  \frac{d\dot{x}}{dz}  \frac{dz}{dt} = \left\{-2\left[ a(1-z) +b z\right] + 2(1-2z)\left[ b-a\right]\right\}\dot{z}^2$$
$$\displaystyle \frac{d^2y}{dt^2} =  \frac{d\dot{y}}{dz}  \frac{dz}{dt} = \left\{-2\left[ c(1-z) +d z\right] + 2(1-2z)\left[ d-c\right]\right\}\dot{z}^2$$
$$z = \displaystyle \frac{t - t_0}{t_1-t_0}, \quad  \dot{z} = \frac{1}{t_1-t_0}$$

### Example

Assume you want the spline that connects the points (1,-1) with (3,4).
Also assume that the derivative at (1,-1) is given by $<1,-3>$ and at
(3,4) is given by $<0,2>$.

We can take $t_0=0$ and $t_1 = 1$. This gives $z = t$, $\dot{z} = 1$,
$a = 1 - 2 = -1$, $b = 2$, $c = -8$, $d = 3$. This gives us the two
splines for the parametric description of the curve:
$$x(t) = (1-t) + 3t + t(1-t)[-1(1-t) + 2t] = -3 t^3+4 t^2+t+1$$
$$y(t) = -(1-t) + 4t + t(1-t)[-8(1-t)+3t] = -11 t^3+19 t^2-3 t-1$$
$$\dot{x} = -9t^2+8t+1 \quad,\quad  \dot{y} = -33t^2 +38t -3$$

### Example

$$\begin{array}{l}
v = \sqrt{(-9t^2+8t+1)^2 + ( -33t^2 +38t -3)^2}\\[3mm]
\kappa =   \displaystyle  \frac{(-9t^2+8t+1)( -66t+38) - ( -33t^2 +38t -3)( -18t+8)}{v^3}\\[3mm]
\dot{\phi_1} = \displaystyle \frac{v}{r}\left(\kappa L \pm 1\right) \\[3mm]
\dot{\phi_2} = \displaystyle \frac{v}{r}\left(-\kappa L \pm 1\right)
\end{array}$$

### Spline Example

In [ ]:
t0, t1 = 0, 1
x0, y0 = 1, -1
x1, y1 = 3, 4
xd0 , yd0 = 1, -3
xd1 = 0
yd1 = 2
dt = (t1-t0)
dx = (x1-x0)
dy = (y1-y0)
a = xd0*dt- dx
b = -xd1*dt+dx
c = yd0*dt-dy
d = -yd1*dt+dy

In [ ]:
N = 100
t = range(t0,t1,length=N)
dotz = 1.0/dt
z = @. (dotz)*(t-t0)
x = @. (1-z)*x0 + z*x1+z*(1-z)*(a*(1-z)+b*z)
y = @. (1-z)*y0 + z*y1+z*(1-z)*(c*(1-z)+d*z)
ptx = [x0,x1]
pty = [y0,y1]
plot(x,y,legend=false, title = "Cubic Interpolant", xlims=(0,4), ylims=(-2,5))
p=scatter!(ptx,pty)
println("Done")

In [ ]:
display(p)

### Example

In [ ]:
dotx = @. (dx+(1-2*z)*(a*(1-z)+b*z)+z*(1-z)*(b-a))*dotz
doty = @. (dy+(1-2*z)*(c*(1-z)+d*z)+z*(1-z)*(d-c))*dotz
ddotx = @. (-2*(a*(1-z)+b*z)+2*(1-2*z)*(b-a))*dotz
ddoty = @. (-2*(c*(1-z)+d*z)+2*(1-2*z)*(d-c))*dotz

r = 1.0
L = 4.0
v = @. sqrt(dotx*dotx + doty*doty)
kappa =  @. (dotx*ddoty - doty*ddotx)/(v*v*v)
dotphi1 = @. (v/r)*(kappa*L +1)
dotphi2 = @. (v/r)*(-kappa*L+1)

plot(t,dotphi1)
p = plot!(t,dotphi2, title = "Wheel Speeds")
println("Done")

In [ ]:
display(p)

In [ ]:
r = 1.0
L = 4.0
xp = zeros(N)
yp = zeros(N)
th = zeros(N)
xp[1] = 1
yp[1] = -1
th[1] = atan(-3,1)
Δt = (t1-t0)/N
δ = (r*Δt/2.0)
for i in 1:N-1
    xp[i+1] = xp[i] + δ*(dotphi1[i]+dotphi2[i])*cos(th[i])
    yp[i+1] = yp[i] + δ*(dotphi1[i]+dotphi2[i])*sin(th[i])
    th[i+1] = th[i] + δ*(dotphi1[i]-dotphi2[i])/L
end
plot(xp,yp, legend = false)
ptx = [x0,x1]
pty = [y0,y1]
plot(xp,yp,legend=false, title = "Path", xlims=(0,4), ylims=(-2,5))
p = scatter!(ptx,pty)
println("Done")

In [ ]:
display(p)

The reality is that errors crop up and one will miss ...

[Run the following]

In [ ]:
function addpath!(N, c, p, col)
    t0, t1 = 0, 1
    x0, y0 = c[1], c[2]
    x1, y1 = c[3], c[4]
    xd0 ,yd0 = c[5], c[6]
    xd1, yd1 = c[7], c[8]
    dt = (t1-t0)
    dx = (x1-x0)
    dy = (y1-y0)
    a = xd0*dt- dx
    b = -xd1*dt+dx
    c = yd0*dt-dy
    d = -yd1*dt+dy
    t = range(t0,t1,length=N)
    dotz = 1.0/dt
    z = @. (dotz)*(t-t0)
    x = @. (1-z)*x0 + z*x1+z*(1-z)*(a*(1-z)+b*z)
    y = @. (1-z)*y0 + z*y1+z*(1-z)*(c*(1-z)+d*z)
    ptx = [x0,x1]
    pty = [y0,y1]
    ax = [x1-0.1,x1]
    ay = [y1,y1]
    plot!(x,y, color = col)
    scatter!(ptx,pty)
    plot!(ax, ay,arrow=true,color=:black,linewidth=2,label="")
    return p
end

N = 20
c = zeros(8)
p = plot(legend=false, linewidth = 2, axis = false, ticks = false, title = "Trajectories", xlims=(-3,3), ylims=(-0.25,5))

c = [0, 1, 1, 2, 1, 0, 1, 0]
p = addpath!(N, c, p, "blue")
c = [1, 2, 2, 1, 1, 0, 1, 0]
p = addpath!(N, c, p, "blue")

c = [0, 2, 1, 3.3, 1, 0, 1, 0]
p = addpath!(N, c, p, "green")
c = [1, 3, 2, 2, 1, 0, 1, 0]
p = addpath!(N, c, p, "blue")

c = [0, 3, 1, 4.3, 1, 0, 1, 0]
p = addpath!(N, c, p, "green")
c = [1, 4.3, 2, 3, 1, 0, 1, 0]
p = addpath!(N, c, p, "red")
annotate!(-1.5,1,"Planned Path")
annotate!(-1.5,2,"Actual Path")
annotate!(-1.5,3,"Corrected Path")
println("Plots generated")

In [ ]:
plot(p)

### Discrete Inverse Kinematics

Recall the discrete forward kinematics $$\begin{array}{l}
 x_{k+1} = x_k + \frac{r\Delta t}{2} (\omega_{1, k}+\omega_{2, k})\cos(\theta_k) \\[5mm]
y_{k+1} = y_k + \frac{r\Delta t}{2} (\omega_{1, k}+\omega_{2, k})\sin(\theta_k) \\[5mm]
\theta_{k+1} = \theta_k + \frac{r\Delta t}{2L} (\omega_{1, k}-\omega_{2, k})
\end{array}$$

### Discrete Inverse Kinematics

The inverse kinematics involve both first and second order derivatives.
If you don’t have the expression explicitly, the derivatives can be
approximated: $$\begin{array}{l}
 \displaystyle \dot{x}_k\approx \nabla x_k =  \frac{x_k  - x_{k-1}}{\Delta t}, \quad
 \displaystyle  \dot{y}_k\approx  \nabla y_k =\frac{y_k  - y_{k-1}}{\Delta t} \\[10mm]
 \displaystyle \ddot{x}_k\approx \delta^2 x_k = \frac{x_{k+1}  - 2x_{k} + x_{k-1}}{\Delta t^2}, \quad
 \displaystyle \ddot{y}_k\approx \delta^2 y_k = \frac{y_{k+1}  - 2y_{k}  + y_{k-1}}{\Delta t^2}  .
\end{array}$$

### Discrete Inverse Kinematics

Plugging in the derivatives we then obtain the formulas for the wheel
velocities. In normal applications, the points come in from a path
planning routine, but the controls are discretized. Meaning you will
have discrete values for time. You can use the formulas at $t=t_k$ for
the points between the interpolating points: $$\boxed{
\begin{array}{l}
v_k = \sqrt{\dot{x}(t_k)^2 + \dot{y}(t_k)^2} , \quad\quad
\displaystyle  \kappa_k = \frac{\dot{x}(t_k) \ddot{y}(t_k ) -  \dot{y}(t_k) \ddot{x}(t_k)}{v_k^3}, \\[3mm]
\displaystyle  \omega_{1,k} = \frac{v_k}{r}(\kappa_k L + 1), \quad\quad
\displaystyle  \omega_{2,k} = \frac{v_k}{r}(-\kappa_k L + 1)
\end{array} }$$

## Simulation with noise

In [ ]:
using Random, Distributions, Plots
Random.seed!(123) # Setting the seed
μ1 = 1.0
σ1 = 0.5
d1 = Normal(μ1, σ1)
μ2 = 2.0
σ2 = 1.0
d2 = Normal(μ2, σ2)
x = rand(d1,100)
y = rand(d2,100)
p1=scatter(x,y,ratio=1)

error = rand(Normal(μ1, σ1),100)
x = LinRange(0,5,100)
y = 2 .* x.+1.0 .+ error
p2 = scatter(x,y,ratio=1)
p=plot(p1,p2, layout=(1,2))
println("Done")

In [ ]:
display(p)

### An example

In [ ]:
using Distributions, Plots, StatsPlots

function wheels(t)
   if (t < 1.5)
       w1 = 1.0
       w2 = 1.0
       return w1, w2
   end
   if (t < 3)
       w1 = 2.0
       w2 = 1.0
       return w1, w2
   end
   w1 = 1.0
   w2 = 1.0
   return w1, w2
end

In [ ]:
using Distributions, Plots, StatsPlots

r = 20.0
l = 12.0
N = 10
dt = 0.01
Tend = 5
NumP = Int(Tend/dt)

μ1, σ1 = 0.0, 0.05
μ2, σ2 = 0.0, 0.01
d1 = Normal(μ1, σ1)
d2 = Normal(μ2, σ2)
tp = LinRange(0,Tend,NumP)
x, y = 0, 0
xp  = zeros(NumP)
yp = zeros(NumP)
thp = zeros(NumP)
xpath  = zeros(N,NumP)
ypath = zeros(N,NumP)
thpath = zeros(N,NumP)
xpts = zeros(N)
ypts = zeros(N)
println("Arrays set")

In [ ]:
x, y = 0, 0
th = 0.0
for i = 2:NumP
    w1,w2 = wheels(tp[i])
    dx = (r*dt/2.0)*(w1+w2)*cos(th)
    dy = (r*dt/2.0)*(w1+w2)*sin(th)
    dth = (r*dt/(2.0*l))*(w1-w2) 
    x = x + dx
    y = y + dy
    th = th + dth
    xp[i] = x
    yp[i] = y
    thp[i] = th
end

In [ ]:
for k = 1:N
    th = 0.0
    x = 0.0
    y = 0.0
    errx = rand(d1,NumP)
    erry = rand(d1,NumP)
    errth = rand(d2,NumP)
    for i = 2:NumP
        w1,w2 = wheels(tp[i])
        dx = (r*dt/2.0)*(w1+w2)*cos(th) + errx[i]
        dy = (r*dt/2.0)*(w1+w2)*sin(th) + erry[i]
        dth = (r*dt/(2.0*l))*(w1-w2) + errth[i]
        x = x + dx
        y = y + dy
        th = th + dth
        xpath[k,i] = x
        ypath[k,i] = y
        thpath[k,i] = th
    end
    xpts[k] = x
    ypts[k] = y
end

In [ ]:
plot(0,0,legend=false)
for j=1:N
    plot!(xpath[j,:], ypath[j,:], legend=false)
end

p = plot!(title="Paths with noise")
display(p)

The following will plots the error ellipse (not complete yet)

In [ ]:
A = [xpts ypts]
cmat = cov(A)
mx = mean(xpts)
my = mean(ypts)
covellipse([mx,my], cmat, n_std=2)
plot!(xp,yp, legend=false)
p = scatter!(xpts, ypts)
println("Done")

In [ ]:
display(p)

## System Architecture

A robot is built from

- Sensors
- Sensor fusion and Mapping
- Planning and Control
- Actuation

All of these run at different speeds (frequencies)

There is not one good loop frequency that will work.

##  Modular design

Good design means one uses different functions, classes,  modules for different software components.

Problems:

- Some aspects of the system are synchronous and some is asynchronous.
- Different components have frequencies and interrupt semantics.
- Multiple processors are often required.
- Processors need not live in the same address space.
- The world requires fault tolerance.
- Software and hardware is constantly changing.
- Multiple languages are useful.


## Interprocess Communication

Tasks can communicate via

- Monolithic architecture via function calls
- Shared file system
- Shared memory
- Messages (via SVIPC or similar)
- Sockets

### Sockets 

For modular, asynchronous, OS and language agnostic communication - sockets are the popular choice.

Low level socket programming (in C) is fast and powerful ...

but can be challenging.   

We will use an wrapper library which hides the details.

Called ZeroMQ or 0MQ

## Zero MQ

ZeroMQ (also known as ØMQ, 0MQ, or zmq) looks like an embeddable networking library but acts like a concurrency framework. It gives you sockets that carry atomic messages across various transports like in-process, inter-process, TCP, and multicast. 

You can connect sockets N-to-N with patterns like fan-out, pub-sub, task distribution, and request-reply. It’s fast enough to be the fabric for clustered products. 

Its asynchronous I/O model gives you scalable multicore applications, built as asynchronous message-processing tasks. 

## Goal 1

Goal one is to communicate from one process to another.  We can shoot for a sequence as shown:

In [ ]:
using GraphRecipes, Plots
x = [1,2,3]
y = [1,1,1]
ax = [.25,.75, NaN, 1.25, 1.75, NaN, 2.25, 2.75]
ay = [1,1, NaN, 1, 1, NaN, 1, 1]
plot(xlims=(0,4), ylims = (0,2), legend = false, ticks = false, axis = false, title="Data Flow")
scatter!(x,y, shape=:rect, ms = 30)
#plot!(ax, ay,arrow=true,color=:black,linewidth=2,label="")
p = plot!(ax,ay, arrow=true, arrowsize=0.5, color=:black)
println("Plot done")

In [ ]:
display(p)

## Goal 2

Goal two is bidirectional communication.

In [ ]:
x = [1,2,3]
y = [1,1,1]
ax = [1.25, 1.75, NaN, 2.25, 2.75]
ay = [.9, .9, NaN, .9, .9]
bx = [1.75, 1.25, NaN, 2.75, 2.25]
by = [1.1, 1.1, NaN, 1.1, 1.1]
plot(xlims=(0,4), ylims = (0,2), legend = false, ticks = false, axis = false, title="Data Flow")
scatter!(x,y, shape=:rect, ms = 30)
plot!(ax,ay, arrow=true, arrowsize=0.5, color=:black)
p = plot!(bx,by, arrow=true, arrowsize=0.5, color=:black)
println("Plot done")

In [ ]:
display(p)

## Goal 3

Many to many communication

In [ ]:
x = [1,2,3, 1, 3]
y = [1,1,1, 2.25, 2.25]
ax = [1.25, 1.75, NaN, 2.25, 2.75, NaN, 1, 1, NaN, 1.25, 1.75, NaN, 1.25, 2.75, NaN, 2.75, 2.25, NaN, 3, 3]
ay = [1, 1, NaN, 1, 1, NaN, 1.4, 1.85, NaN, 2.2, 1.2, NaN, 2.2, 2.2, NaN, 2.2, 1.2, NaN, 1.85, 1.4]
plot(xlims=(0,4), ylims = (0,4), legend = false, ticks = false, axis = false, title="Data Flow")
scatter!(x,y, shape=:rect, ms = 30)
p = plot!(ax,ay, arrow=true, arrowsize=0.5, color=:black)
println("Plot done")

In [ ]:
display(p)

## ZMQ  Request - Reply  Server - One to One

## ZMQ Request - Reply Client

### Robotics Example

### Robotics example cont.

## ZMQ Publisher  - One to many

## ZMQ Subscriber

### Environment

### Map Representation

1.  Map precision vs. application

2.  Features precision vs. map precision

3.  Precision vs. computational complexity

-   Continuous representation

-   Decomposition (discretization)

### Representation of the Environment

Environment Representation  

-   Continuous Metric $\to x, y, \theta$  

-   Discrete Metric $\to$ metric grid  

-   Discrete Topological $\to$ topological grid

### Representation of the Environment

Environmental Modeling

-   Raw sensor data, e.g. laser range data, grayscale images

    -   large volume of data, low distinctiveness on the level of
        individual values

    -   makes use of all acquired information

-   Low level features, e.g. line other geometric features

    -   medium volume of data, average distinctiveness

    -   filters out the useful information, still ambiguities

-   High level features, e.g. doors, a car, Mt Rushmore

    -   low volume of data, high distinctiveness

    -   filters out the useful information, few/no ambiguities, not
        enough information

### Representing the World - Obstacles

In [ ]:
function collide(center1, r1, center2, r2)
    x1 = center1[1]
    y1 = center1[2]
    x2 = center2[1]
    y2 = center2[2]
    d = sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))-r1-r2
    return d
end

### Representing the World - Collision

In [ ]:
using Plots

function build(r, h, k)
    t = range(0,π, length=60)
    hx = r*cos.(t) .+ h
    hy = r*sin.(t) .+ k
    ly = -r*sin.(t) .+ k
    return hx, hy, ly
end

function graph!(hx,hy,ly, col, p)
    p = plot!(hx, ly, fillrange = hy, fillalpha = 0.35, color=col)
    p = plot!(hx,hy, color=:black)
    p = plot!(hx,ly, color=:black)
    return p
end

hx1, hy1, ly1 = build(2,4,5)
hx2, hy2, ly2 = build(2,7,7.6)

p = plot(xlims = (0,10), ylims = (0,10), legend = false, aspect_ratio = :equal, title = "Collision", axis = false, ticks=false)
p = graph!(hx1,hy1,ly1, :blue, p)
p = graph!(hx2,hy2,ly2, :red, p)
println("Plot Done")

In [ ]:
display(p)

### Representing the World - Collision Sensing

In [ ]:
t = range(0,2*π, length=60)
s = range(0,2*π, length=24)
x = cos.(t).+2
y = sin.(t).+2
xx = cos.(s).+2
yy = sin.(s).+2
p = plot(xlims = (0.5,3.5), ylims = (0.5,3.5), legend = false, aspect_ratio = :equal, title = "Collision", axis = false, ticks=false)
plot!(x,y)
p = scatter!(xx,yy, ms = 10)
println("Plot Done")

In [ ]:
display(p)

In [ ]:
t = range(0,2*π, length=60)
s = range(0,2*π, length=24)
x = cos.(t).+2
y = sin.(t).+2
xx = cos.(s).+2
yy = sin.(s).+2
u = sqrt.(t).-0.55
p = plot(xlims = (0.25,3.5), ylims = (0.25,3.5), legend = false, aspect_ratio = :equal, title = "Collision and Sensing", axis = false, ticks=false)
p = plot!(x,y, color=:black)
p = scatter!(xx,yy, ms = 10, color=:blue)
p = plot!(t, u)
p = plot!([2.35,2],[1,2],arrow=true,color=:black,linewidth=2,label="")
p = plot!([2,2.85],[2,2.3],arrow=true,color=:black,linewidth=2,label="")
println("Plot Done")

In [ ]:
display(p)

### Representing the World - Collision Sensing

Using the normal vector, $\hat{n} = <n_1, n_2>$, the tangent to the
boundary is computed via $$T = \pm <n_2, -n_1>$$ where the sign is taken
so that motion is to the right (right hand rule).

### LIDAR

At each angle or direction, the lidar unit projects a laser beam out. It
receives the reflected signal and computes the distance. Naively one
simply measures the time of flight, divides by two (for the round trip)
and multiplies by $c$ (the speed of light): $D = RT$. This provides the
distance of the nearest obstacle at the current angle.

### How is this done?

Using a two colored image, let white be free space and red or black
indicate occupied space. To simulate the beam out of the LIDAR, create a
virtual line out of the lidar and follow a straight line along white
pixels until you run into a colored pixel. Stop at the first colored
pixel. Using the endpoints of the line segment (virtual lidar to object
pixel), the distance can be computed. Let $(n,m)$ be the start of the
line and let $(i,j)$ be the location of the object pixel and recall the
distance is $d = \sqrt{(i-n)^2 + (j-m)^2}$

### LIDAR Simulation Algorithm

Algorithm